# packages used

In [5]:
%matplotlib inline

# misc
from IPython.display import display, HTML
import numpy as np

# DATA - prep
#kaggle
import pandas as pd
import sklearn.model_selection

# ML - models 
import sklearn.linear_model
import sklearn.tree
import sklearn.ensemble
import xgboost

# hypertune
from sklearn.model_selection import GridSearchCV

# ML - accuracy
import sklearn.metrics

# Plot and visualize
# import matplotlib.pyplot as plt
# import shap

# savemodel 
import joblib

# Get data

Setup:
- follow "API credential step" listed here: https://github.com/Kaggle/kaggle-api
    - go to https://www.kaggle.com/ (login)
    - go to my_profile (download kaggle.json)
    - put it in ~/.kaggle/kaggle.json
    - `cp ~/Downloads/kaggle.json ~/.kaggle/kaggle.json`
    - `chmod 600 ~/.kaggle/kaggle.json`
- Go to kaggle and join competition: 
    - https://www.kaggle.com/c/titanic
- install kaggle
- download data
- profit!!!

In [6]:

metadata = {
    'basepath' : 'data/',
    'dataset':'titanic',
    'train' : 'train.csv',
    'test' : 'test.csv'}

In [3]:
# make folder
# download .zip
# unzip
# remove the .zip
# (data is placed ../data/titanic)

# !mkdir -p {metadata['basepath']}
# !kaggle competitions download -c dataset {metadata['dataset']} -p {metadata['basepath']}
# !unzip -o {metadata['basepath']}{metadata['dataset']}.zip -d {metadata['basepath']}{metadata['dataset']}/
# !rm {metadata['basepath']}{metadata['dataset']}.zip

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 17.8MB/s]
Archive:  data/titanic.zip
  inflating: data/titanic/gender_submission.csv  
  inflating: data/titanic/test.csv   
  inflating: data/titanic/train.csv  


# Load and explore

In [7]:
# load
train = pd.read_csv("{basepath}/{dataset}/{train}".format(**metadata))
test = pd.read_csv("{basepath}/{dataset}/{test}".format(**metadata))


# Simple imputation + cleaning

In [8]:
def clean(df):
    dfc = df.copy()
    
    # Simple map
    dfc['Sex'] = dfc['Sex'].map({"female":0,"male":1}).astype(int)
    
    # simple Impute
    dfc['Age'] = dfc["Age"].fillna(-1)
    dfc['Fare'] = dfc["Fare"].fillna(-1)

    # Simple feature engineering (combining two variables)
    dfc['FamilySize'] = dfc['SibSp'] + dfc['Parch'] + 1

    # Simple feature engineering (converting to boolean)
    dfc['Has_Cabin'] = dfc["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    dfc = dfc.drop(["Cabin"],axis=1)
    
    # "Stupid feature engineering - apply length 
    dfc['Name_length'] = dfc['Name'].apply(len)
    dfc = dfc.drop(["Name"],axis=1)
    dfc['Ticket_length'] = dfc['Ticket'].apply(len)
    dfc = dfc.drop(["Ticket"],axis=1)
    
    # 1-hot encoding - different options are encoded as booleans
    # ie. 1 categorical - become 3: 0-1 features.
    dfc['Embarked_Q'] = dfc['Embarked'].apply(lambda x: 1 if x=="Q" else 0)
    dfc['Embarked_S'] = dfc['Embarked'].apply(lambda x: 1 if x=="S" else 0)
    dfc['Embarked_C'] = dfc['Embarked'].apply(lambda x: 1 if x=="C" else 0)
    dfc = dfc.drop(["Embarked"],axis=1)
    
    
    return dfc


clean_train = clean(train)
clean_test = clean(test)


In [7]:
#display(pd.DataFrame(clean_test.isna().mean() ,columns=["is na fraction"]))


In [10]:
target = "Survived"

y = clean_train[target]
X = clean_train.drop([target],axis=1)

# Split data in train and validation
seed = 42
test_size = 0.7

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
    X,
    y,
    random_state = seed,
    test_size = test_size)


# ML

In [10]:
# xgboost
model_xgboost = xgboost.sklearn.XGBClassifier()
model_xgboost.fit(X_train, y_train);



# Hyper parameter tuning

## xgboost

In [11]:
##### from sklearn.model_selection import GridSearchCV

xgb_param_grid = {
        'n_estimators' : [100,200,300],
        'max_depth': [2,3,5,8],
        'min_sample_split': [2,3,5,8],
        "learning_rate": [0.05,0.1,0.5]
}


model_xgboost_dummy = xgboost.sklearn.XGBClassifier(min_sample_split=3,learning_rate=0.1)
grid_search_xgb = GridSearchCV(estimator = model_xgboost_dummy, param_grid = xgb_param_grid , 
                          cv = 5, n_jobs = -1, verbose = 2)

grid_search_xgb.fit(X_train, y_train)


Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 393 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 705 out of 720 | elapsed:   13.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:   13.5s finished
/Users/epedersen/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1,
       min_sample_split=3, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 300], 'max_depth': [2, 3, 5, 8], 'min_sample_split': [2, 3, 5, 8], 'learning_rate': [0.05, 0.1, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [12]:
best_grid = grid_search_xgb.best_params_
display(best_grid)
model_xgboost_best = xgboost.sklearn.XGBClassifier(**best_grid)
model_xgboost_best.fit(X_train,y_train);



{'learning_rate': 0.05,
 'max_depth': 5,
 'min_sample_split': 2,
 'n_estimators': 100}

# Eval ML

In [16]:

models = {
    "model_xgboost_best":model_xgboost_best
}

for name,model in zip(models.keys(),models.values()):
    acc = sklearn.metrics.accuracy_score(
     y_true = y_val,
     y_pred = model.predict(X_val)
    )
    acc_train = sklearn.metrics.accuracy_score(
     y_true = y_train,
     y_pred = model.predict(X_train)
    )
    
    print(name," | train: ",round(acc_train,3)," | test: ",round(acc,3))



model_xgboost_best  | train:  0.959  | test:  0.812


# save the model

In [23]:
# save the model to disk

filename =  metadata["basepath"] + 'model_xgboost_best.sav'
joblib.dump(model_xgboost_best, filename)
 
# some time later...
 
# load the model from disk
#loaded_model = joblib.load(filename)



['data/model_xgboost_best.sav']

In [ ]:
# old predict for kaggle


# passengerid
# id = "PassengerId"
# out = pd.DataFrame(data = test[id], columns = [id])

# # target
# out_target = model_xgboost_best.predict(clean_test)
# out[target] = pd.DataFrame(out_target
#                           ,columns = [target]
#                           ,dtype=np.int32
#                           )


# # put them out
# outfile = metadata["basepath"] + "output_xgboost.csv"
# out.to_csv(path_or_buf = outfile,
#            index = False)

